In [13]:
import requests as req
import pandas as pd
from pprint import pprint
from functools import reduce

pd.options.display.max_rows = 400

In [10]:
# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/widget"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
api_response = res.json()["data"]

pprint(api_response[0], depth=2)

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html

def extract_metadata(wgets_on_api, wget, attributes):
    atts = wget["attributes"]
    wgets_on_api[atts["slug"]] = {}
    for att in attributes:
        try:
            wgets_on_api[atts["slug"]][att] = atts[att]
        except:
            wgets_on_api[atts["slug"]][att] = None
            
    return wgets_on_api

attributes = ['protected', 'published', 'queryUrl', 'dataset', 'default']
widgets_on_api = reduce(lambda agg, elem: extract_metadata(agg, elem, attributes), api_response, {})

    
# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
widgets_on_api = pd.DataFrame.from_dict(widgets_on_api, orient='index')

{'attributes': {'application': [...],
                'authors': '',
                'createdAt': '2017-03-22T14:39:41.070Z',
                'dataset': '118311ed-4b1b-4e75-bdc7-636f644de0c2',
                'default': True,
                'defaultEditableWidget': False,
                'description': 'This data set distributed by the World Bank '
                               'from the WHO/UNICEF Joint Monitoring Programme '
                               '(JMP) for Water Supply and Sanitation shows '
                               'the percent of the population with access to '
                               'improved drinking water sources in 2015.  The '
                               'improved drinking water source includes piped '
                               'water on premises (piped household water '
                               'connection located inside the user’s dwelling, '
                               'plot or yard), and other improved drinking '
                 

In [12]:
widgets_on_api.to_csv('widgets_on_api_feb_27_2018.csv')

In [14]:
widgets_on_api

,protected,published,queryUrl,dataset,default
0910-through-Explore,False,False,None,e663eb09-04de-4f39-b871-35c6c2ed10b5,False
10-Largest-Political-Boundary-Areas,False,True,,098b33df-6871-4e53-a5ff-b56a7d989f9a,False
10-strongest-recent-earthquakes-in-mexico,False,False,None,63f34231-7369-4622-81f1-28a144d17835,False
1010-map,False,True,None,52c55378-0484-48c3-92fc-3ee94d21c716,False
111,False,True,,d639909f-bcf3-4875-b8c3-35f030b68ed3,False
1209-3,False,False,None,d8a45b34-4cc0-42f4-957d-e13b37e9182e,False
12345,False,False,,093b89c8-58e6-4b0c-bec1-d46170f7a253,False
15-strongest-recent-quakes-in-Mexico,False,False,None,63f34231-7369-4622-81f1-28a144d17835,False
1908-7,False,False,None,16df8ada-87cc-4907-adce-a98bc4e91856,False
1915-2014-Cold-Water-Coral-Depth-points-m,False,True,,6b8442f5-4766-4444-94b4-d6676277fd80,True
